## Homework 2

### 1.
Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both `Bachelor semestre 1` and `Bachelor semestre 6`. Compute how many months it took each student to go from the first to the sixth semester. Partition the data between male and female students, and compute the average -- is the difference in average statistically significant?

In [52]:
import requests
import pandas as pd
import html5lib
from bs4 import BeautifulSoup

#ISA base url. This is the page that displays the form
isa_base_url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html'

#list of ID of academic periods, starting from 2007-2008
academic_periods = [978181, 978187, 978195, 39486325, 123455150, 123456101, 213637754, 213637922, 213638028, 355925344]

params_dict = {'ww_x_GPS': -1,
               'ww_i_reportmodel': 133685247,
               #'ww_c_langue': None,
               'ww_i_reportModelXsl': 133685270,
               #'zz_x_UNITE_ACAD': 'Informatique',
               'ww_x_UNITE_ACAD': 249847,
               #'zz_x_PERIODE_ACAD': '2007-2008',
               #'ww_x_PERIODE_ACAD': 978181,
               #'zz_x_PERIODE_PEDAGO': 'Bachelor+semestre+1',
               'ww_x_PERIODE_PEDAGO': 249108,
               #'zz_x_HIVERETE': None,
               'ww_x_HIVERETE': 'null',
               #'dummy': 'ok'
              }
resp = requests.get(isa_base_url, params=params_dict)
data = resp.text
soup = BeautifulSoup(data, 'html.parser')
#print(soup.prettify())
table = soup.findAll('table')[0]
#print(len(table))
rows = table.findAll('th')
rows
#print(rows[4])
#data = {''}


[<th colspan="12"><font color="black">Informatique, 2004-2005, Bachelor semestre 1</font>
  (107 ét.)
     </th>,
 <th>Civilité</th>,
 <th>Nom Prénom</th>,
 <th>Orientation Bachelor</th>,
 <th>Orientation Master</th>,
 <th>Spécialisation</th>,
 <th>Filière opt.</th>,
 <th>Mineur</th>,
 <th>Statut</th>,
 <th>Type Echange</th>,
 <th>Ecole Echange</th>,
 <th>No Sciper</th>,
 <th colspan="12"><font color="black">Informatique, 2005-2006, Bachelor semestre 1</font>
  (95 ét.)
     </th>,
 <th>Civilité</th>,
 <th>Nom Prénom</th>,
 <th>Orientation Bachelor</th>,
 <th>Orientation Master</th>,
 <th>Spécialisation</th>,
 <th>Filière opt.</th>,
 <th>Mineur</th>,
 <th>Statut</th>,
 <th>Type Echange</th>,
 <th>Ecole Echange</th>,
 <th>No Sciper</th>,
 <th colspan="12"><font color="black">Informatique, 2006-2007, Bachelor semestre 1</font>
  (87 ét.)
     </th>,
 <th>Civilité</th>,
 <th>Nom Prénom</th>,
 <th>Orientation Bachelor</th>,
 <th>Orientation Master</th>,
 <th>Spécialisation</th>,
 <th>Filiè